# Instalation library



In [1]:
%tensorflow_version 1.x
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

!pip install -q keras
import keras
!pip install gensim

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.models import Model
from keras import regularizers

import numpy as np
import pandas as pd
!pip install nltk
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
import string

import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

import math
from sklearn.metrics import mean_squared_error

from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 

import matplotlib.pyplot as plt
import timeit

TensorFlow 1.x selected.
Instructions for updating:
non-resource variables are not supported in the long term


Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Pembagian training, test, and validation

In [3]:
# train_data=pd.read_csv('/content/drive/My Drive/input/1429_1.csv')
fold_1=pd.read_csv('/content/drive/My Drive/input/Amazon_1.csv')
fold_2=pd.read_csv('/content/drive/My Drive/input/Amazon_2.csv')
fold_3=pd.read_csv('/content/drive/My Drive/input/Amazon_3.csv')
fold_4=pd.read_csv('/content/drive/My Drive/input/Amazon_4.csv')
fold_5=pd.read_csv('/content/drive/My Drive/input/Amazon_5.csv')

frames=[fold_2,fold_5,fold_3,fold_4]
data = pd.concat(frames, ignore_index=True) #concate

In [4]:
train_data=data
test_data= fold_1
# test_data= fold_2
# test_data= fold_3
# test_data= fold_4
# test_data=fold_5

In [5]:
val_data=train_data.sample(frac=0.2499849588,random_state=200)#valdata 
train_data=train_data.drop(val_data.index)
# val_data=fold_4
print(len(val_data))
print(len(train_data))

6925
20776


In [6]:
star=train_data.reviewsrating.unique()
dic={}
print(star)
for i,stars in enumerate(star):
    dic[stars]=i
labels=train_data.reviewsrating.apply(lambda x:dic[x])
labels_val=val_data.reviewsrating.apply(lambda x:dic[x])

[5. 4. 1. 3. 2.]


In [7]:
star=train_data.reviewsrating.value_counts()
star

5.0    14309
4.0     5111
3.0      887
2.0      237
1.0      232
Name: reviewsrating, dtype: int64

# Resampling

In [8]:
from sklearn.utils import resample
x_train = train_data['reviewstext']
y_train = train_data['reviewsrating']

data =pd.concat([x_train, y_train], axis=1)
class_1= data[data['reviewsrating']== 1]
class_2= data[data['reviewsrating']== 2]
class_3= data[data['reviewsrating']== 3]
class_4= data[data['reviewsrating']== 4]
class_5= data[data['reviewsrating']== 5]
oversample_1 = resample(class_1, 
                       replace=True, 
                       n_samples=len(class_5), 
                       random_state=42)
oversample_2 = resample(class_2, 
                       replace=True, 
                       n_samples=len(class_5), 
                       random_state=42)
oversample_3 = resample(class_3, 
                       replace=True, 
                       n_samples=len(class_5), 
                       random_state=42)
oversample_4 = resample(class_4, 
                       replace=True, 
                       n_samples=len(class_5), 
                       random_state=42)
train_data= pd.concat([class_5, oversample_1,oversample_2,oversample_3,oversample_4])

train_data.reviewsrating.value_counts()

1.0    14309
3.0    14309
4.0    14309
2.0    14309
5.0    14309
Name: reviewsrating, dtype: int64

# Preprocessing

In [9]:
reviews=train_data.reviewstext
rev=[] #remove punctuation
for i in range(len(reviews)):
  review=reviews.iloc[i]
  rev.append(review.translate(str.maketrans('', '', string.punctuation)))

stopwords = nltk.corpus.stopwords.words('english') #set and add stopwords list
tokens_without_sw=[] #Stopords removal
for i in range(len(rev)):
  tokenized_word=word_tokenize(rev[i])
  tokens_lw=[word.lower() for word in tokenized_word]
  tokens_sw =[word for word in tokens_lw if not word in stopwords]
  filtered_sentence = (" ").join(tokens_sw)
  tokens_without_sw.append(filtered_sentence)
train_data=train_data.assign(text=tokens_without_sw)

In [10]:
train_data['reviewstext']=train_data['reviewstext'].astype('str')
val_data['reviewstext']=val_data['reviewstext'].astype('str')
test_data['reviewstext']=test_data['reviewstext'].astype('str')
reviews=train_data.reviewstext

In [11]:
#Process Tokenizer
NUM_WORDS=2000
tokenizer = Tokenizer(num_words=NUM_WORDS,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'',
                      lower=True)
tokenizer.fit_on_texts(reviews)
sequences_train = tokenizer.texts_to_sequences(reviews)
sequences_valid=tokenizer.texts_to_sequences(val_data.reviewstext)
sequences_test=tokenizer.texts_to_sequences(test_data.reviewstext)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 11011 unique tokens.


In [12]:
X_train = pad_sequences(sequences_train)
X_val = pad_sequences(sequences_valid,maxlen=X_train.shape[1])
X_test = pad_sequences(sequences_test,maxlen=X_train.shape[1])
y_train = to_categorical(np.asarray(labels[train_data.index]))
y_val = to_categorical(np.asarray(labels_val[val_data.index]))
print('Shape of X train, X validation, X test tensor:', X_train.shape,X_val.shape,X_test.shape)
print('Shape of label train, validation, test tensor:', y_train.shape,y_val.shape)

Shape of X train, X validation, X test tensor: (71545, 786) (6925, 786) (6925, 786)
Shape of label train, validation, test tensor: (71545, 5) (6925, 5)


In [13]:


word_vectors = KeyedVectors.load_word2vec_format('/content/drive/My Drive/input/GoogleNews-vectors-negative300.bin', binary=True)

EMBEDDING_DIM=300
vocabulary_size=min(len(word_index)+1,NUM_WORDS)
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in word_index.items():
    if i>=NUM_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

del(word_vectors)

from keras.layers import Embedding
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


# Convolutional Neural Network

In [14]:

sequence_length = X_train.shape[1]
filter_sizes = [3,4,5]
num_filters = 100
drop = 0.5

In [15]:
start = timeit.default_timer()
inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)
reshape = Reshape((sequence_length,EMBEDDING_DIM,1))(embedding)

conv_0 = Conv2D(num_filters, (filter_sizes[0], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_1 = Conv2D(num_filters, (filter_sizes[1], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_2 = Conv2D(num_filters, (filter_sizes[2], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)


maxpool_0 = MaxPooling2D((sequence_length - filter_sizes[0] + 1, 1), strides=(1,1))(conv_0)
maxpool_1 = MaxPooling2D((sequence_length - filter_sizes[1] + 1, 1), strides=(1,1))(conv_1)
maxpool_2 = MaxPooling2D((sequence_length - filter_sizes[2] + 1, 1), strides=(1,1))(conv_2)


merged_tensor = concatenate([maxpool_0, maxpool_1, maxpool_2], axis=1)
flatten = Flatten()(merged_tensor)
reshape = Reshape((3*num_filters,))(flatten)
dropout = Dropout(drop)(reshape)
output = Dense(units=5, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout)

# this creates a model that includes
model = Model(inputs, output)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [16]:
del(sequences_train)
del(sequences_valid)
del(reviews)

In [17]:
adam = Adam(lr=1e-3)

model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['acc'])
callbacks = [EarlyStopping(monitor='val_loss')]
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 786)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 786, 300)     3303600     input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 786, 300, 1)  0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 782, 1, 100)  150100      reshape_1[0][0]                  
____________________________________________________________________________________________

In [18]:
history= model.fit(X_train, y_train, batch_size=50, epochs=10, verbose=1, validation_data=(X_val, y_val),callbacks=callbacks)


Train on 71545 samples, validate on 6925 samples
Epoch 1/10
71545/71545 [==============================] - 262s 4ms/step - loss: 1.1018 - acc: 0.6859 - val_loss: 1.1953 - val_acc: 0.6312
Epoch 2/10
71545/71545 [==============================] - 257s 4ms/step - loss: 0.7462 - acc: 0.8091 - val_loss: 1.1417 - val_acc: 0.6315
Epoch 3/10
71545/71545 [==============================] - 257s 4ms/step - loss: 0.6515 - acc: 0.8387 - val_loss: 1.1726 - val_acc: 0.5847


In [19]:
y_pred=model.predict(X_test)
stop = timeit.default_timer()
print('Time: ', stop - start) 

Time:  789.8119259019999


In [20]:
to_submit=pd.DataFrame(index=test_data.reviewsid,data={'1':y_pred[:,dic[1]],
                                                '2':y_pred[:,dic[2]],
                                                '3':y_pred[:,dic[3]],
                                                '4':y_pred[:,dic[4]],
                                                '5':y_pred[:,dic[5]],
                                                })

In [21]:
to_submit.to_csv('hasil1_softmaxAmazon.csv')

In [22]:
real= np.array(test_data.reviewsrating)

In [23]:
prediction=[]
for i in range(y_pred.shape[0]):
  rating_pred=np.argmax(y_pred[i])
  if rating_pred == dic[1]:
    rating_pred=1
  elif rating_pred == dic[2]:
    rating_pred=2 
  elif rating_pred == dic[3]:
    rating_pred=3
  elif rating_pred == dic[4]:
    rating_pred=4
  else :
    rating_pred=5
  prediction.append(rating_pred)

In [24]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(real,prediction)
print(accuracy)

0.5776173285198556


In [25]:
results = confusion_matrix(real, prediction) #Perhitungan F1 score dengan kelas {1,2,3,4,5}
print ('Confusion Matrix :')
print(results) 
print ('Accuracy Score :', accuracy_score(real, prediction)) 
print ('Report : ')
print (classification_report(real, prediction))

Confusion Matrix :
[[  21   11   39   18    9]
 [  12    8   36   21   16]
 [  12   11  119  126   40]
 [  14   32  205  884  574]
 [  31   34  187 1497 2968]]
Accuracy Score : 0.5776173285198556
Report : 
              precision    recall  f1-score   support

         1.0       0.23      0.21      0.22        98
         2.0       0.08      0.09      0.08        93
         3.0       0.20      0.39      0.27       308
         4.0       0.35      0.52      0.42      1709
         5.0       0.82      0.63      0.71      4717

    accuracy                           0.58      6925
   macro avg       0.34      0.37      0.34      6925
weighted avg       0.66      0.58      0.60      6925



In [26]:

mse=mean_squared_error(real, prediction)
print(mse)
rmse=math.sqrt(mse)
print(rmse)

0.7471480144404332
0.8643772408158565
